In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader

# Import from your codebase
from CLAPWrapper import CLAPWrapper
from datasets.esc50 import ESC50

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cpu


In [ ]:
print("\n" + "="*80)
print("Loading ESC50 Dataset")
print("="*80)

root_path = "../data"
dataset = ESC50(root=root_path, download=False)
print(f"✅ Dataset loaded: {len(dataset)} samples")
print(f"   Classes: {len(dataset.classes)} categories")
print(f"   Sample classes: {dataset.classes[:5]}")

# Prepare text prompts
prompt = 'this is the sound of '
text_labels = [prompt + x for x in dataset.classes]
print(f"\n📝 Text prompts: {len(text_labels)} classes")


Loading ESC50 Dataset
Loading audio files


0it [00:00, ?it/s]

2000it [00:00, 15630.79it/s]

✅ Dataset loaded: 2000 samples
   Classes: 50 categories
   Sample classes: ['airplane', 'breathing', 'brushing teeth', 'can opening', 'car horn']

📝 Text prompts: 50 classes


In [ ]:
# ============================================================================
# STEP 3: Initialize Models
# ============================================================================
print("\n" + "="*80)
print("Initializing Models")
print("="*80)

# Residual config con pc_weights = 1.0 (identità)
residual_config = {
    'mode': 'attention',
    'n_components_ratio': 1.0,
    'reweight_factor': 0.0,
    'target_layers': [0, 1, 2, 3],  # Layers dove applicare reweighting
}

print("\n🔧 Loading CLAP Standard...")
clap_standard = CLAPWrapper(
    version='2023',  # or '2022'
    use_cuda=torch.cuda.is_available(),
    type='classic'
)

print("\n🔧 Loading ResiDualCLAP...")
clap_residual = CLAPWrapper(
    version='2023',
    use_cuda=torch.cuda.is_available(),
    type='residual',
    residual_config=residual_config
)


Initializing Models

🔧 Loading CLAP Standard...



🔧 Loading ResiDualCLAP...

🔧 Setup ResiDual HTSAT
Modalità: ATTENTION
Target layers: [0, 1, 2, 3]
PCA components ratio: 1.0
Reweight factor: 0.0

✓ layer_0:
  Modalità: PER-HEAD reweighting
  Num blocks: 2
  Heads per block: 4
  Total heads: 8
  Head dim: 24D → 24 PCs

✓ layer_1:
  Modalità: PER-HEAD reweighting
  Num blocks: 2
  Heads per block: 8
  Total heads: 16
  Head dim: 24D → 24 PCs

✓ layer_2:
  Modalità: PER-HEAD reweighting
  Num blocks: 6
  Heads per block: 16
  Total heads: 96
  Head dim: 24D → 24 PCs

✓ layer_3:
  Modalità: PER-HEAD reweighting
  Num blocks: 2
  Heads per block: 32
  Total heads: 64
  Head dim: 24D → 24 PCs




In [ ]:
# Prepare audio samples for PCA fitting
print("Collecting samples for PCA fitting (max 200 samples)...", end='')

# Create a simple dataloader wrapper per PCA fitting
class SimpleAudioDataset:
    def __init__(self, wrapper, esc50_dataset, max_samples=1000):
        self.wrapper = wrapper
        self.audio_paths = []
        for i in range(min(max_samples, len(esc50_dataset))):
            audio_path, _, _ = esc50_dataset[i]
            self.audio_paths.append(audio_path)
    
    def __len__(self):
        return len(self.audio_paths)
    
    def __getitem__(self, idx):
        audio_tensor = self.wrapper.load_audio_into_tensor(
            self.audio_paths[idx],
            self.wrapper.args.duration,
            resample=True
        )
        # ✅ Assicurati sia 1D
        if audio_tensor.dim() > 1:
            audio_tensor = audio_tensor.squeeze()
        
        return audio_tensor

# Create dataset and loader
pca_dataset = SimpleAudioDataset(clap_residual, dataset, max_samples=50)
pca_loader = DataLoader(
    pca_dataset, 
    batch_size=16, 
    shuffle=False,
    num_workers=0,  # Start with 0 for debugging
    pin_memory=False
)
print("OK")

In [ ]:
pca_dataset[0].shape

torch.Size([308700])

In [ ]:
list(pca_loader)[1].shape

torch.Size([16, 308700])

In [ ]:
len(dataset)

2000

In [ ]:
print("\n" + "="*80)
print("Fitting PCA Components")
print("="*80)

# Fit PCA
print(f"Fitting PCA on {len(pca_dataset)} samples...")
variance_ratios = clap_residual.clap.audio_encoder.base.htsat.fit_spectral_layers(
    pca_loader,
    max_samples=50
)

print("\n📊 PCA Variance Ratios:")
print("="*80)

# Gestione corretta della struttura gerarchica
for layer_name, layer_data in variance_ratios.items():
    print(f"\n{layer_name}:")
    
    # Controlla se è mode 'attention' (struttura annidata) o 'layer' (array diretto)
    if isinstance(layer_data, dict):
        # Mode ATTENTION: layer_data contiene blocchi
        for block_name, block_data in layer_data.items():
            print(f"  {block_name}:")
            
            # block_data contiene le teste
            for head_name, head_variance in block_data.items():
                # head_variance è l'array numpy con le variance ratios
                top3 = head_variance[:3]
                cumulative = head_variance[:3].sum()
                print(f"    {head_name}: top3={[f'{v:.3f}' for v in top3]}, cum={cumulative:.3f}")
    else:
        # Mode LAYER: layer_data è direttamente l'array di variance ratios
        top5 = layer_data[:5]
        cumulative = layer_data[:5].sum()
        print(f"  Top 5 components: {[f'{v:.4f}' for v in top5]}")
        print(f"  Cumulative variance (top 5): {cumulative:.4f}")


Fitting PCA Components
Fitting PCA on 50 samples...

📊 Fitting Spectral Layers
Modalità: ATTENTION
Target layers: [0, 1, 2, 3]
Max samples: 50

📦 Fase 1: Raccolta hidden states...


Raccolta:   0%|          | 0/4 [00:00<?, ?it/s]


🔍 DEBUG: Processing layer_0 in ATTENTION mode with fitting
  Block 0: input shape = torch.Size([16, 4096, 96])
    After norm1: torch.Size([16, 4096, 96]), is_contiguous=True
    After view to 4D: torch.Size([16, 64, 64, 96])
    After shift: torch.Size([16, 64, 64, 96])
    After window_partition: torch.Size([1024, 8, 8, 96]), is_contiguous=True
    After view for attention: torch.Size([1024, 64, 96])
    Calling attention with collect_for_fitting=True
      🔍 WindowAttentionReweighting.forward: x shape = torch.Size([1024, 64, 96]), collect=True
      After QKV: q=torch.Size([1024, 4, 64, 24]), k=torch.Size([1024, 4, 64, 24]), v=torch.Size([1024, 4, 64, 24])
      After attn scores: torch.Size([1024, 4, 64, 64])
      After attn @ v: x_heads shape = torch.Size([1024, 4, 64, 24]), is_contiguous=True
      Calling _collect_heads...
      🔍 _collect_heads: x_heads shape = torch.Size([1024, 4, 64, 24]), is_contiguous=True
        Head 0: shape=torch.Size([1024, 64, 24]), is_contiguous=Fa

Raccolta:  25%|██▌       | 1/4 [00:03<00:10,  3.42s/it, samples=16]

    After FFN: torch.Size([16, 64, 768])

🔍 DEBUG: Processing layer_0 in ATTENTION mode with fitting
  Block 0: input shape = torch.Size([16, 4096, 96])
    After norm1: torch.Size([16, 4096, 96]), is_contiguous=True
    After view to 4D: torch.Size([16, 64, 64, 96])
    After shift: torch.Size([16, 64, 64, 96])
    After window_partition: torch.Size([1024, 8, 8, 96]), is_contiguous=True
    After view for attention: torch.Size([1024, 64, 96])
    Calling attention with collect_for_fitting=True
      🔍 WindowAttentionReweighting.forward: x shape = torch.Size([1024, 64, 96]), collect=True
      After QKV: q=torch.Size([1024, 4, 64, 24]), k=torch.Size([1024, 4, 64, 24]), v=torch.Size([1024, 4, 64, 24])
      After attn scores: torch.Size([1024, 4, 64, 64])
      After attn @ v: x_heads shape = torch.Size([1024, 4, 64, 24]), is_contiguous=True
      Calling _collect_heads...
      🔍 _collect_heads: x_heads shape = torch.Size([1024, 4, 64, 24]), is_contiguous=True
        Head 0: shape=tor

...

    After FFN: torch.Size([16, 64, 768])

🔍 DEBUG: Processing layer_0 in ATTENTION mode with fitting
  Block 0: input shape = torch.Size([16, 4096, 96])
    After norm1: torch.Size([16, 4096, 96]), is_contiguous=True
    After view to 4D: torch.Size([16, 64, 64, 96])
    After shift: torch.Size([16, 64, 64, 96])
    After window_partition: torch.Size([1024, 8, 8, 96]), is_contiguous=True
    After view for attention: torch.Size([1024, 64, 96])
    Calling attention with collect_for_fitting=True
      🔍 WindowAttentionReweighting.forward: x shape = torch.Size([1024, 64, 96]), collect=True
      After QKV: q=torch.Size([1024, 4, 64, 24]), k=torch.Size([1024, 4, 64, 24]), v=torch.Size([1024, 4, 64, 24])
      After attn scores: torch.Size([1024, 4, 64, 64])
      After attn @ v: x_heads shape = torch.Size([1024, 4, 64, 24]), is_contiguous=True
      Calling _collect_heads...
      🔍 _collect_heads: x_heads shape = torch.Size([1024, 4, 64, 24]), is_contiguous=True
        Head 0: shape=tor

Raccolta:  75%|███████▌  | 3/4 [00:10<00:03,  3.35s/it, samples=48]

      Final output: torch.Size([16, 64, 768])
    After attention: torch.Size([16, 64, 768])
    After view for reverse: torch.Size([16, 8, 8, 768])
    After window_reverse: torch.Size([16, 8, 8, 768])
    After view back to 3D: torch.Size([16, 64, 768])
    After FFN: torch.Size([16, 64, 768])

🔍 DEBUG: Processing layer_0 in ATTENTION mode with fitting
  Block 0: input shape = torch.Size([2, 4096, 96])
    After norm1: torch.Size([2, 4096, 96]), is_contiguous=True
    After view to 4D: torch.Size([2, 64, 64, 96])
    After shift: torch.Size([2, 64, 64, 96])
    After window_partition: torch.Size([128, 8, 8, 96]), is_contiguous=True
    After view for attention: torch.Size([128, 64, 96])
    Calling attention with collect_for_fitting=True
      🔍 WindowAttentionReweighting.forward: x shape = torch.Size([128, 64, 96]), collect=True
      After QKV: q=torch.Size([128, 4, 64, 24]), k=torch.Size([128, 4, 64, 24]), v=torch.Size([128, 4, 64, 24])
      After attn scores: torch.Size([128, 4,

Raccolta: 100%|██████████| 4/4 [00:10<00:00,  2.65s/it, samples=50]

        After reshape: torch.Size([128, 24])
        Head 25: shape=torch.Size([2, 64, 24]), is_contiguous=False
        After reshape: torch.Size([128, 24])
        Head 26: shape=torch.Size([2, 64, 24]), is_contiguous=False
        After reshape: torch.Size([128, 24])
        Head 27: shape=torch.Size([2, 64, 24]), is_contiguous=False
        After reshape: torch.Size([128, 24])
        Head 28: shape=torch.Size([2, 64, 24]), is_contiguous=False
        After reshape: torch.Size([128, 24])
        Head 29: shape=torch.Size([2, 64, 24]), is_contiguous=False
        After reshape: torch.Size([128, 24])
        Head 30: shape=torch.Size([2, 64, 24]), is_contiguous=False
        After reshape: torch.Size([128, 24])
        Head 31: shape=torch.Size([2, 64, 24]), is_contiguous=False
        After reshape: torch.Size([128, 24])
      Applying reweighting to 32 heads...
      Final output: torch.Size([2, 64, 768])
    After attention: torch.Size([2, 64, 768])
    After view for reverse: tor

In [ ]:
# Get text embeddings ONCE for all classes
text_embeddings = clap_standard.get_text_embeddings(text_labels)
print(f"   Text embeddings shape: {text_embeddings.shape}")

# Test on subset first (use full dataset later)
test_size = 100  # Start with 200 samples for speed
print(f"\n📊 Testing on {test_size} samples...")

y_preds_baseline, y_labels = [], []

for i in tqdm(range(test_size), desc="Baseline"):
    # Get audio file path and label
    audio_path, target, one_hot_target = dataset[-(i+1)]
    
    # Get audio embedding
    audio_embedding = clap_standard.get_audio_embeddings([audio_path], resample=True)
    
    # Compute similarity
    similarity = clap_standard.compute_similarity(audio_embedding, text_embeddings)
    
    # Get prediction
    y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()
    y_preds_baseline.append(y_pred)
    y_labels.append(one_hot_target.detach().cpu().numpy())

# Calculate accuracy
y_labels_array = np.concatenate(y_labels, axis=0)
y_preds_baseline_array = np.concatenate(y_preds_baseline, axis=0)

baseline_acc = accuracy_score(
    np.argmax(y_labels_array, axis=1), 
    np.argmax(y_preds_baseline_array, axis=1)
)

print(f"\n✅ Baseline Accuracy: {baseline_acc:.3f} ({baseline_acc*100:.1f}%)")

   Text embeddings shape: torch.Size([50, 1024])

📊 Testing on 100 samples...


Baseline:   0%|          | 0/100 [00:00<?, ?it/s]


✅ Baseline Accuracy: 0.920 (92.0%)


In [ ]:
# Get text embeddings ONCE for all classes
text_embeddings = clap_residual.get_text_embeddings(text_labels)
print(f"   Text embeddings shape: {text_embeddings.shape}")

# Test on subset first (use full dataset later)
test_size = 100  # Start with 200 samples for speed
print(f"\n📊 Testing on {test_size} samples...")

y_preds_residual, y_labels = [], []

for i in tqdm(range(test_size), desc="residual"):
    # Get audio file path and label
    audio_path, target, one_hot_target = dataset[-(i+1)]
    
    # Get audio embedding
    audio_embedding = clap_residual.get_audio_embeddings([audio_path], resample=True)
    
    # Compute similarity
    similarity = clap_residual.compute_similarity(audio_embedding, text_embeddings)
    
    # Get prediction
    y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()
    y_preds_residual.append(y_pred)
    y_labels.append(one_hot_target.detach().cpu().numpy())

# Calculate accuracy
y_labels_array = np.concatenate(y_labels, axis=0)
y_preds_residual_array = np.concatenate(y_preds_residual, axis=0)

baseline_acc = accuracy_score(
    np.argmax(y_labels_array, axis=1), 
    np.argmax(y_preds_residual_array, axis=1)
)

print(f"\n✅ Residual Accuracy: {baseline_acc:.3f} ({baseline_acc*100:.1f}%)")

   Text embeddings shape: torch.Size([50, 1024])

📊 Testing on 100 samples...


residual:   0%|          | 0/100 [00:00<?, ?it/s]

      🔍 WindowAttentionReweighting.forward: x shape = torch.Size([64, 64, 96]), collect=False
      After QKV: q=torch.Size([64, 4, 64, 24]), k=torch.Size([64, 4, 64, 24]), v=torch.Size([64, 4, 64, 24])
      After attn scores: torch.Size([64, 4, 64, 64])
      After attn @ v: x_heads shape = torch.Size([64, 4, 64, 24]), is_contiguous=True
      Applying reweighting to 4 heads...
      Final output: torch.Size([64, 64, 96])
      🔍 WindowAttentionReweighting.forward: x shape = torch.Size([64, 64, 96]), collect=False
      After QKV: q=torch.Size([64, 4, 64, 24]), k=torch.Size([64, 4, 64, 24]), v=torch.Size([64, 4, 64, 24])
      After attn scores: torch.Size([64, 4, 64, 64])
      After attn @ v: x_heads shape = torch.Size([64, 4, 64, 24]), is_contiguous=True
      Applying reweighting to 4 heads...
      Final output: torch.Size([64, 64, 96])
      🔍 WindowAttentionReweighting.forward: x shape = torch.Size([16, 64, 192]), collect=False
      After QKV: q=torch.Size([16, 8, 64, 24]), k

In [ ]:
audio_path, target, one_hot_target = dataset[0]

In [ ]:
clap_residual.get_audio_embeddings([audio_path], resample=True)

      🔍 WindowAttentionReweighting.forward: x shape = torch.Size([64, 64, 96]), collect=False
      After QKV: q=torch.Size([64, 4, 64, 24]), k=torch.Size([64, 4, 64, 24]), v=torch.Size([64, 4, 64, 24])
      After attn scores: torch.Size([64, 4, 64, 64])
      After attn @ v: x_heads shape = torch.Size([64, 4, 64, 24]), is_contiguous=True
      Applying reweighting to 4 heads...
      Final output: torch.Size([64, 64, 96])
      🔍 WindowAttentionReweighting.forward: x shape = torch.Size([64, 64, 96]), collect=False
      After QKV: q=torch.Size([64, 4, 64, 24]), k=torch.Size([64, 4, 64, 24]), v=torch.Size([64, 4, 64, 24])
      After attn scores: torch.Size([64, 4, 64, 64])
      After attn @ v: x_heads shape = torch.Size([64, 4, 64, 24]), is_contiguous=True
      Applying reweighting to 4 heads...
      Final output: torch.Size([64, 64, 96])
      🔍 WindowAttentionReweighting.forward: x shape = torch.Size([16, 64, 192]), collect=False
      After QKV: q=torch.Size([16, 8, 64, 24]), k

tensor([[ 0.7623,  0.2343, -0.5101,  ...,  1.8940,  0.9414, -0.0119]])

In [ ]:
clap_standard.get_audio_embeddings([audio_path], resample=True)

tensor([[ 0.7623,  0.2343, -0.5101,  ...,  1.8940,  0.9414, -0.0119]])